In [13]:
import tensorflow as tf
from keras import Model
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image

In [43]:
# Define paths for training and testing datasets
train_path = './data/training_set/'
test_path = './data/test_set/'

# Image dimensions and batch size
img_width, img_height = 128, 128
batch_size = 32

# Create image data generators for training and testing
#data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    # rotation_range=20,      # Random rotation within the range [-20, 20] degrees
    # width_shift_range=0.1,  # Random horizontal shift by up to 10% of the width
    # height_shift_range=0.1, # Random vertical shift by up to 10% of the height
    # shear_range=0.2,        # Shear intensity (shear angle in counter-clockwise direction in radians)
    # zoom_range=0.2,         # Random zoom up to 20%
    # horizontal_flip=True,   # Randomly flip inputs horizontally
    # fill_mode='nearest'     # Strategy used for filling in newly created pixels
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    color_mode="grayscale"
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode='binary',
    shuffle=False  # Keep data in the same order as the directory
)


Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [44]:
# Define the CNN model
# model = Sequential([
#     Input(shape=(img_width, img_height, 1)),
#     Flatten(),
#     Dense(128, activation='relu'),  # Increasing the number of neurons in Dense layer
#     Dense(64, activation='relu'),   # Adding another Dense layer
#     Dense(2, activation='softmax')
# ])

inputs = keras.Input(shape=(img_width, img_height,1))
x = Flatten()(inputs)
x = Dense(units=128, activation='relu')(x)
x = Dense(units=64, activation='relu')(x)
x = Dense(units=64, activation='relu')(x)
x = Dense(units=32, activation='relu')(x)
outputs = Dense(units=1, activation='sigmoid')(x)

custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.000001)
model = Model(inputs=inputs, outputs=outputs)
# Compile the model
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "functional_30"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_15 (Flatten)            │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 128)            │     2,097,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_65 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_66 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_67 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_68 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,111,809 (8.06 MB)

 Trainable params: 2,111,809 (8.06 MB)

 Non-trainable params: 0 (0.00 B)

In [45]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
)

Epoch 1/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 19s 70ms/step - accuracy: 0.4998 - loss: 0.6961 - val_accuracy: 0.5180 - val_loss: 0.6922
Epoch 2/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 18s 71ms/step - accuracy: 0.5371 - loss: 0.6893 - val_accuracy: 0.5571 - val_loss: 0.6876
Epoch 3/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 18s 70ms/step - accuracy: 0.5689 - loss: 0.6819 - val_accuracy: 0.5675 - val_loss: 0.6841
Epoch 4/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 18s 69ms/step - accuracy: 0.5772 - loss: 0.6788 - val_accuracy: 0.5541 - val_loss: 0.6821
Epoch 5/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 18s 70ms/step - accuracy: 0.5951 - loss: 0.6752 - val_accuracy: 0.5615 - val_loss: 0.6805
Epoch 6/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 17s 68ms/step - accuracy: 0.5914 - loss: 0.6739 - val_accuracy: 0.5803 - val_loss: 0.6782
Epoch 7/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 16s 64ms/step - accuracy: 0.5949 - loss: 0.6732 - val_accuracy: 0.5798 - val_loss: 0.6773
Epoch 8/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 16s 63ms/step - accuracy: 0.6030 - loss: 0.6662 - 